<a href="https://colab.research.google.com/github/Givi-Modebadze/ML_Final_Project/blob/main/experiments/N-Beats_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'ML_Final_Project/walmart-recruiting-store-sales-forecasting'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv')
features = pd.read_csv('features.csv')
df = df.merge(stores, how='left', left_on='Store', right_on='Store')
df = df.merge(features, how='left', left_on=['Store','Date'], right_on=['Store','Date'])
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [ ]:
!pip install pytorch-forecasting pytorch-lightning torchmetrics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

In [ ]:
df['Weekly_Sales'].fillna(df['Weekly_Sales'].median(), inplace=True)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
for col in markdown_cols:
    if col in df.columns:
        df[col].fillna(0, inplace=True)
        test_df[col].fillna(0, inplace=True)

In [ ]:
numeric_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
for col in numeric_cols:
    if col in df.columns:
        df[col].fillna(df[col].median(), inplace=True)
        test_df[col].fillna(test_df[col].median(), inplace=True)

In [ ]:
df = df.sort_values(['Store', 'Dept', 'Date'])
df['time_idx'] = df.groupby(['Store', 'Dept']).cumcount()

In [ ]:
df['group_id'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)

In [ ]:
df['log_sales'] = np.log(df['Weekly_Sales'] + 1)

In [ ]:
min_length = 20
group_counts = df['group_id'].value_counts()
valid_groups = group_counts[group_counts >= min_length].index
df_filtered = df[df['group_id'].isin(valid_groups)].copy()

In [ ]:
df_filtered = df_filtered.sort_values(['Store', 'Dept', 'Date'])
df_filtered['time_idx'] = df_filtered.groupby(['Store', 'Dept']).cumcount()

In [ ]:
max_prediction_length = 8
max_encoder_length = 24

In [ ]:
training = TimeSeriesDataSet(
    df_filtered[lambda x: x.time_idx <= x.time_idx.max() - max_prediction_length],
    time_idx="time_idx",
    target="Weekly_Sales",
    group_ids=["group_id"],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Store", "Dept", "Type"],
    static_reals=["Size"],
    time_varying_known_categoricals=["IsHoliday"],
    time_varying_known_reals=["Temperature", "Fuel_Price", "CPI", "Unemployment"] + markdown_cols,
    time_varying_unknown_reals=["Weekly_Sales"],
    target_normalizer=None,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

validation = TimeSeriesDataSet.from_dataset(
    training,
    df_filtered,
    predict=True,
    stop_randomization=True
)

In [ ]:
batch_size = 64
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*2, num_workers=0)

In [ ]:
def weighted_mae_loss(y_pred, y_true, is_holiday_mask=None):
    mae = torch.abs(y_pred - y_true)
    if is_holiday_mask is not None:
        weights = torch.where(is_holiday_mask, 5.0, 1.0)
        mae = mae * weights
    return torch.mean(mae)

In [ ]:
model = NBeats.from_dataset(
    training,
    learning_rate=0.001,
    hidden_size=256,
    num_blocks=[3, 3, 3],
    num_block_layers=4,
    expansion_coefficient_lengths=[3, 7, 3],
    sharing=[True, True, False],
    stack_types=["trend", "seasonality", "generic"],
    loss=MAE(),
    log_interval=10,
    reduce_on_plateau_patience=3,
    weight_decay=1e-2,
)

In [ ]:
trainer = pl.Trainer(
    max_epochs=50,
    accelerator="auto",
    devices=1,
    gradient_clip_val=1.0,
    limit_train_batches=0.5,
    enable_checkpointing=True,
    logger=True,
    log_every_n_steps=10
)

In [ ]:
trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
predictions = model.predict(val_dataloader, return_y=True)
predictions_df = pd.DataFrame({
    'y_true': predictions.output.numpy().flatten(),
    'y_pred': predictions.prediction.numpy().flatten()
})

In [ ]:
val_mae = np.mean(np.abs(predictions_df['y_true'] - predictions_df['y_pred']))
print(val_mae)